#### Gradient Boosting

While in RandomForest a lot of trees are built independently of each other in gradient boosting trees are built sequentially.

**Algorithm**

Input: Loss function $L$, Data $(x_i, y_i), i=1...N$, number of estimators built $M$

**1. Initialize**

$f_0(x)= argmin_\gamma \sum_{i=1}^N L(y_i, \gamma)$       # this means that the first estimator estimates the same constant $\gamma$ for all data points

**2. For m=1 to $M$:**

a) For $i=1,..N$ compute $r_{im}=-\left[\frac {\partial L(y_i, f(x_i))}{\partial f(x_i)}\right]_{f=f_{m-1}}$        # "in which direction do we need to improve to minimize the loss in x_i"

b) Fit a regression tree to $(x_i, r_{im})$ with regions $R_{jm}$ $j=1, J_m$, # Capture the improvement directions for all data-points in a regression tree

c) For $j=1,..., J_m$ compute the improvement $\gamma_{jm}= argmin_\gamma \sum_{x_i\in R_{jm}} L(y_i, f_{m-1}(x_i)+\gamma)$  # for each region of the 'improvement regression tree' fit a constant improvement

d) Update $f_m (x) = f_{m-1}(x)+\sum_{j=1}^{J_m} \gamma_{jm} 1(x\in R_{jm})$

**3. Output** $f_M$

In the following we will work with XGBoost which is an implementation of gradient boosting with additional features focused on performance and speed.

#### Coding

Setup:

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('melb_data.csv')

# Select subset of predictors
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# Select target
y = data.Price

# Separate data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [7]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
my_model.fit(X_train, y_train)

from sklearn.metrics import mean_absolute_error

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

Mean Absolute Error: 240970.94236238956


#### Parameter Tuning of XGBoost

https://xgboost.readthedocs.io/en/stable/python/python_api.html#module-xgboost.sklearn

`n_estimators`
n_estimators specifies how many times to go through the modeling cycle described above. It is equal to the number of models that we include in the ensemble.


`early_stopping_rounds¶`
early_stopping_rounds offers a way to automatically find the ideal value for n_estimators. Early stopping causes the model to stop iterating when the validation score stops improving, even if we aren't at the hard stop for n_estimators. It's smart to set a high value for n_estimators and then use early_stopping_rounds to find the optimal time to stop iterating.
Since random chance sometimes causes a single round where validation scores don't improve, you need to specify a number for how many rounds of straight deterioration to allow before stopping. Setting early_stopping_rounds=5 is a reasonable choice. I


`learning_rate¶`
Instead of getting predictions by simply adding up the predictions from each component model, we can multiply the predictions from each model by a small number (known as the learning rate) before adding them in. This means each tree we add to the ensemble helps us less. So, we can set a higher value for n_estimators without overfitting. If we use early stopping, the appropriate number of trees will be determined automatically. In general, a small learning rate and large number of estimators will yield more accurate XGBoost models, though it will also take the model longer to train since it does more iterations through the cycle.


`n_jobs¶`
On larger datasets where runtime is a consideration, you can use parallelism to build your models faster. It's common to set the parameter n_jobs equal to the number of cores on your machine. On smaller datasets, this won't help. The resulting model won't be any better, so micro-optimizing for fitting time is typically nothing but a distraction. But, it's useful in large datasets where you would otherwise spend a long time waiting during the fit command.

In [24]:
my_model = XGBRegressor(n_estimators=1000, early_stopping_rounds=5,learning_rate=0.2, n_jobs=4)
my_model.fit(X_train, y_train,
             eval_set=[(X_valid, y_valid)],
             verbose=False)                     # If verbose is True and an evaluation set is used, the evaluation metric measured on the validation set is printed to stdout at each boosting stage.
predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

Mean Absolute Error: 240812.90228276877
